In [ ]:
# Week08 Assignment: CNN with Keras

# Getting Started
import os
import numpy as np

from keras.models import Model
from keras.layers import Dense, Activation,Flatten, Conv2D, MaxPooling2D
from keras.layers import Input, ZeroPadding2D, BatchNormalization, Add, AveragePooling2D
from keras import backend as K
from keras.initializers import glorot_uniform
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

In [ ]:
#1 데이터 편집
K.set_image_data_format('channels_last')
path = 'C:/Users/YY/Documents/Data/Dataset/train'

# 데이터 늘리기(Rock 데이터 수)
# Augmentation tool 선언
image_gen = ImageDataGenerator(
#featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

In [ ]:
def augment(generator):
"""
    :param generator: ImageDataGenerator
    :return: 변형된 10배수의 사진들이 저장된다.
    """
    classes_list = os.listdir(path)

for class_name in classes_list:
        img_list = os.listdir(os.path.join(path, class_name))

for img_name in img_list:
# 각각의 이미지의 path name을 정의하고 로드 후 array로 변환한다.
            img_path_name = os.path.join(path, class_name, img_name)
            img = image.load_img(img_path_name)
            img = image.img_to_array(img)

# generator에 넣기 위해 rank를 4로 맞춰준다.
            img = img.reshape((1,) + img.shape)

            i = 0

            for batch in generator.flow(img, batch_size=1, save_to_dir=os.path.join(path, class_name),
                save_prefix=class_name, save_format='jpg'):
# 각각의 이미지에 대해 10번만 수행한다.
                i += 1
                if i > 10:
break

augment(image_gen)

In [ ]:
# 케라스 loading 메서드를 쓰는 것이 속도가 훨씬 좋습니다.
def LoadPreprocess(path):

    dataset = []
    y = []

# classes_list: path내에 있는 폴더명을 리스트로 만듦
    classes_list = os.listdir(path)

for class_number, class_name in enumerate(classes_list):
# img_list: 'rock.jpg'과 같은 이미지명을 담은 리스트임
        img_list = os.listdir(os.path.join(path, class_name))

for img_number, img_name in enumerate(img_list):
# 각각의 이미지의 path name을 정의한다.
            img_path_name = os.path.join(path, class_name, img_name)

# 이미지 로드
            img = image.load_img(img_path_name, target_size=(64, 64))

# 이미지를 np.array로 바꿔줌
            # shape은 (height, width, channels) = (128, 128, 3)
            img_input = image.img_to_array(img)
            dataset.append(img_input)
            y.append(class_number)

    dataset = np.array(dataset)
    y = np.array(y)
    Y = np.eye(3)[y.astype(int)]
    dataset = dataset.astype('float64')
    dataset /= 255
    return dataset, Y

In [ ]:
#2 Block 및 모델 정의
# 모델은 Resnet50
# 효과적인 학습이 가능할 뿐만 아니라 VGG나 inception 계열보다 학습할 Parameter의 양도 적음
# 먼저 2종류의 block을 정의함
def Identity_block(X, kernel, filters, stage, block):
"""
    X -- 인풋 텐서 (examples, height_prev, width_prev, channel_prev)
    kernel -- 커널 사이즈를 나타내는 정수(중간에 한 번 축소를 위해 정의)
    filters -- CONV layer의 필터 수를 담은 리스트
    stage -- 단계를 나타내는 정수
    block -- 단계 내 몇 번째 블록인지 나타내는 문자열

    Returns: X -- output of the identity block, 텐서: (height, width, channel)
    """
    # Setting
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X

# 1st Component
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base+'2a',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

# 2nd Compoenent
    X = Conv2D(filters=F2, kernel_size=(kernel, kernel), strides=(1, 1), padding='same', name=conv_name_base+'2b',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

# 3rd Component
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base+'2c',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

# 위에서 저장한 X_shortcut을 다시 추가해준다.
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
return X

In [ ]:
def Conv_block(X, kernel, filters, stage, block, s=2):
"""
    s -- strides를 나타냄(중간에 한 번 축소를 위해 정의)

    Returns: X -- output of the convolutional block, 텐서: (height, width, channel)
    """
    # Setting
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X

# 1st Component
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), name=conv_name_base+'2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)

# 2nd Compoenent
    X = Conv2D(filters=F2, kernel_size=(kernel, kernel), strides=(1, 1), name=conv_name_base+'2b', padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)

# 3rd Component
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), name=conv_name_base+'2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2c')(X)

# Shortcut에도 1X1 conv를 적용해 준다.
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), name=conv_name_base+'1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base+'1')(X_shortcut)

# 위에서 저장한 X_shortcut을 다시 추가해준다.
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
return X

In [ ]:
class CNN(Model):
def __init__(self, in_shape=(128, 128, 3), classes=3):
self.in_shape = in_shape
self.classes = classes
self.BuildModel()
super().__init__(self.X_input, self.Y)
self.compile()


def BuildModel(self):
        in_shape = self.in_shape
        classes = self.classes
        X_input = Input(shape=in_shape)
        X = ZeroPadding2D((3, 3))(X_input)

# Stage 1
        X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), name='conv1',
                   kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name='bn_conv1')(X)
        X = Activation('relu')(X)
        X = MaxPooling2D((3, 3), strides=(2, 2))(X)

# Stage 2
        X = Conv_block(X, kernel=3, filters=[64, 64, 256], stage=2, block='a', s=1)
        X = Identity_block(X, kernel=3, filters=[64, 64, 256], stage=2, block='b')
        X = Identity_block(X, kernel=3, filters=[64, 64, 256], stage=2, block='c')

# Stage 3
        X = Conv_block(X, kernel=3, filters=[128, 128, 512], stage=3, block='a', s=2)
        X = Identity_block(X, kernel=3, filters=[128, 128, 512], stage=3, block='b')
        X = Identity_block(X, kernel=3, filters=[128, 128, 512], stage=3, block='c')
        X = Identity_block(X, kernel=3, filters=[128, 128, 512], stage=3, block='d')

# Stage 4
        X = Conv_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
        X = Identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='b')
        X = Identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='c')
        X = Identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='d')
        X = Identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='e')
        X = Identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='kernel')

# Stage 5
        X = Conv_block(X, kernel=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
        X = Identity_block(X, kernel=3, filters=[512, 512, 2048], stage=5, block='b')
        X = Identity_block(X, kernel=3, filters=[512, 512, 2048], stage=5, block='c')

        X = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', name='avg_pool')(X)
        X = Flatten()(X)
        Y = Dense(units=classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

self.X_input, self.Y = X_input, Y

# Create model
        # model = Model(inputs=X_input, outputs=X, name='ResNet50')


    def compile(self):
        Model.compile(self, optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#4 학습
dataset, Y = LoadPreprocess(path=path)
X_train, X_val, Y_train, Y_val = train_test_split(dataset, Y, test_size=0.1, stratify=Y, random_state=0)

print("X shape: ", X_train.shape)
print("Y shape: ", Y_train.shape)

Resnet = CNN(in_shape=(64, 64, 3), classes=3)
# Resnet.summary()
Resnet.fit(X_train, Y_train, batch_size=32, epochs=5, validation_data=(X_val, Y_val), callbacks=[EarlyStopping(patience=10)])


# 결과: Epoch5가 끝난 이후의 Validation Accuracy는 100%



# Test: 70개의 새로운 이미지
test_path = path = 'C:/Users/YY/Documents/Data/Dataset/test'
X_test, Y_test = LoadPreprocess(path=test_path)
print("X shape: ", X_test.shape)
print("Y shape: ", Y_test.shape)
Y_predicted = Resnet.predict(X_test, batch_size=32)
P = np.argmax(Y_predicted, axis=1)
T = np.argmax(Y_test, axis=1)
np.sum(P == T) / 70

# 정확도는 100%